## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-06-23-51-12 +0000,nyt,Authorities Investigate a New Message in Nancy...,https://www.nytimes.com/2026/02/06/us/savannah...
1,2026-02-06-23-48-56 +0000,nyt,F.B.I. Investigates Links to Biological Labs i...,https://www.nytimes.com/2026/02/06/us/biolab-l...
2,2026-02-06-23-47-23 +0000,bbc,Police probe new message in search for mother ...,https://www.bbc.com/news/articles/c2e1e7nlegko...
3,2026-02-06-23-45-28 +0000,nypost,Mural of murdered Ukrainian refugee that’s loa...,https://nypost.com/2026/02/06/us-news/mural-of...
4,2026-02-06-23-43-27 +0000,nyt,A Pro-Israel Group’s Move Backfires as Gaza Te...,https://www.nytimes.com/2026/02/06/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2452/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
60,trump,38
55,epstein,21
2,new,19
273,iran,14
206,olympics,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
218,2026-02-06-16-02-27 +0000,latimes,"Trump shares, then deletes, racist image of th...",https://www.latimes.com/politics/story/2026-02...,84
363,2026-02-06-01-19-00 +0000,wsj,Trump Calls for New Nuclear Pact With Russia a...,https://www.wsj.com/politics/national-security...,81
52,2026-02-06-22-24-00 +0000,wsj,President Trump posted a video including image...,https://www.wsj.com/politics/trump-obamas-apes...,79
241,2026-02-06-14-24-28 +0000,cbc,Trump shares election conspiracy social media ...,https://www.cbc.ca/news/world/trump-truth-soci...,77
83,2026-02-06-21-25-00 +0000,wsj,Hobbit-Inspired Startup Becomes First New Bank...,https://www.wsj.com/finance/banking/hobbit-ins...,76


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
218,84,2026-02-06-16-02-27 +0000,latimes,"Trump shares, then deletes, racist image of th...",https://www.latimes.com/politics/story/2026-02...
193,72,2026-02-06-17-16-34 +0000,nyt,Milan Winter Olympics Opening Ceremony: What T...,https://www.nytimes.com/2026/02/06/world/europ...
91,68,2026-02-06-21-15-34 +0000,nypost,Nightlife veteran Richie Romero opens three ne...,https://nypost.com/2026/02/06/business/nightli...
102,56,2026-02-06-20-55-59 +0000,wapo,U.S. and Iran hold nuclear talks amid threats ...,https://www.washingtonpost.com/national-securi...
99,54,2026-02-06-21-02-15 +0000,nypost,Pima County Sheriff might have made a catastro...,https://nypost.com/2026/02/06/us-news/pima-cou...
70,51,2026-02-06-21-44-00 +0000,wsj,Bill and Hillary Clinton are once again prepar...,https://www.wsj.com/politics/looming-epstein-d...
312,44,2026-02-06-10-00-47 +0000,wapo,Epstein built ties to Russians and sought to m...,https://www.washingtonpost.com/world/2026/02/0...
202,43,2026-02-06-16-54-39 +0000,nyt,Russian General Is Shot in Latest Attack on a ...,https://www.nytimes.com/2026/02/06/world/europ...
348,42,2026-02-06-04-26-26 +0000,nypost,AOC touts money for her NYC district in fundin...,https://nypost.com/2026/02/05/us-news/aoc-tout...
25,40,2026-02-06-23-05-00 +0000,bbc,Will Sunday's snap election gamble pay off for...,https://www.bbc.com/news/articles/ckgl53zll3jo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
